# SLAM算法介绍

## 1. 名词解释：

### 1.1 什么是SLAM？

SLAM，即Simultaneous localization and mapping，中文可译作“同时定位与地图构建”。它描述的是这样一类过程：机器人在陌生环境中运动，通过处理各类传感器收集的机器人自身及环境信息，精确地获取对机器人自身位置的估计（即“定位”），再通过机器人自身位置确定周围环境特征的位置（即“建图”）

在SLAM过程中，机器人不断地在收集各类传感器信息，如激光雷达的点云、相机的图像、imu的信息、里程计的信息等，通过对这些不断变化的传感器的一系列分析计算，机器人会实时地得出自身行进的轨迹（比如一系列时刻的位姿），但得到的轨迹往往包含很大误差，因此需要进行修正优化，修正的过程很可能不再是实时进行的。实时得出自身行进轨迹的过程一般称作“前端”，修正优化的过程一般称作“后端”。

实现后端优化的处理方法可以分为滤波和优化两类。

### 1.2 什么是滤波？

滤波在一般工程领域指的是根据一定规则对信号进行筛选，保留需要的内容，如各种高通滤波、低通滤波、带通滤波等。但在SLAM算法的语境下，滤波指的是“贝叶斯滤波”概念下的一系列“滤波器”，它们通过概率分析，使用传感器读数、传感器参数、机器人上一时刻位姿等信息，对机器人的下一时刻位姿作出修正：机器人不够准确的粗略轨迹经过”过滤“，变得更准确了。

SLAM中常见滤波有：EKF扩展卡尔曼滤波、UKF无迹卡尔曼滤波、particle filter粒子滤波等。

### 1.3 什么是优化问题？什么是非线性最小二乘优化问题？

各种滤波手段在SLAM问题中曾经占据主导地位，但随着地图规模的扩大（如机器人行进的面积范围增大、引入视觉算法后地图更“精细”），滤波方法所需要的计算量会不断增大。因此现阶段各种优化算法成为了SLAM问题后端处理方法的主流。

什么是优化问题呢？假设有一个函数f，以x为输入，以y为输出，那么一个优化问题就是通过某种手段找到一个x，使y的值最大/最小。而一个SLAM问题的优化中，x通常指的是各种待确定的状态量，比如机器人在各个时刻的位姿、地图中特征点的空间位置等，y通常指的是各种误差，比如传感器测量的量与状态量的差。SLAM问题待优化的函数f通常是非线性的，而且是以二次方项加和的形式存在的，因此属于非线性最小二乘优化问题。

解决非线性优化的开源库如google的Ceres，应用于cartographer、VINS等算法中。

### 1.4 什么是图优化？

图优化指的是把一个优化问题以一个“图”（graph）的形式表示出来，可以用到图论相关的性质和算法，本质上还是一个优化问题。可以简单理解：待优化的状态量，即机器人在各个时刻的位姿、地图中特征点的空间位置，可以表示为graph的各个顶点，相关的顶点间以边连接，各个边代表的就是误差项，所以图优化问题就是通过优化各个顶点的位置，使所有的边加起来的和最小。

解决图优化的开源库如g2o，应用于ORB SLAM等算法中。

### 1.5 什么是约束？

在图优化问题中，顶点与顶点间连接的边就称为一个“约束”（constraint），这个约束可以表示如激光测量量与位置状态量之间的差值、imu测量量与位置状态量之间的差值等。

### 1.6 什么是回环检测

回环检测，也可以称为闭环检测等。简单理解就是，机器人“看到”了看到过的场景，就叫做回环检测成功。回环检测在SLAM问题中，对后端优化具有重要作用。

### 1.7 一个最简单的例子：

[graph slam tutorial : 从推导到应用1](https://heyijia.blog.csdn.net/article/details/47686523)

## 2. 举例分析

主武器与辅助武器：![99A](img/99A.jpeg)

对于一辆坦克来说，炮塔中央的主炮显然就是主武器，其他辅助武器可以有：机枪、反坦克导弹等。

相似地，对于激光slam算法，激光雷达是主武器，imu、里程计等属于辅助武器；对于视觉slam算法，相机就是主武器，imu、里程计等属于辅助武器。


### 2.1 激光slam举例：

cartographer

![主流程](img/cartographer.png)

在SLAM问题的工程实践中，所谓的非线性优化，其实不止出现在后端的全局优化阶段。以google的cartographer为例：

算法前端接收一帧接一帧的激光扫描数据scans，插入到一个小范围的子图（submap）中（比如规定90帧scans组成一个子图），通过调用非线性优化解算库Ceres解决scan在submap中的插入位置问题，在这个优化过程中，imu和里程计负责提供初始值；后端负责进行“回环检测”，寻找新建立的子图submap和之前的scan间的约束，调用非线性优化解算库Ceres计算这个约束，使用一种叫”分支定界“的方法提供这类优化的初始值；最终，后端还要根据约束对所有已有的scan和submap进行全局优化，再次调用非线性优化解算库Ceres解决这个问题。

所以可以粗略地认为，在cartographer中有三处都运用了非线性优化。

### 2.2 视觉slam举例：

VINS-mono

![VINS](img/VINS-mono.jpeg)

港科大的VINS是视觉融合imu信息处理SLAM问题的典范。以单目视觉算法为主的VINS-mono为例：

首先进行”初始化“步骤，在此步骤中，视觉图像和imu信息互相辅助，imu解决了单目图像无法测量深度的问题，并提供了重力方向，视觉图像标定了imu的某些内部参数；

通过”滑窗“方法，使用图像、imu信息建立非线性优化问题，解算每帧图像的优化后位姿，以上内容组成了VIO，即所谓”视觉imu里程计“，可以算是前端的内容，但实际上这个前端也是在使用非线性优化在一直优化每帧的位姿的。

如果回环检测成功检测到了闭环，那么通过非线性优化进行”重定位“，调整滑窗内的位姿；最终通过全局优化，使用非线性优化方法修正所有帧的位姿。

为便于理解，总结一下imu在不同slam算法中的作用：

1. imu在cartographer中的主要作用：通过scan match插入一帧激光建立submap前，预估机器人新位姿，给非线性优化提供初始值。

2. imu在VINS中的主要作用：在“初始化”阶段，获取图像深度尺度等参数；参与VIO优化约束建立。